Takes ingredient table in the database (constucted by stockdb) and adds ingredient information from a usda food resource webpage. Current columns are an id number, a food group which the item belongs, and a short description of the item. 

The items were linked with the usda info using levenstein distance between the ingredient name in the database and 2 different descriptions in the USDA data. There were two promising ways of finding matches, the first was to use the simple description of the item ie: (lentil, LENTIL), and the second was another which tested a more complicated description string: ie (lentil, [canned lentil, raw, descriptor, etc]). In the end, I took both distances and compared their scores. Scores had to be over 75 to be considered. If both were over 75, I took the highest, matched it to the USDA dataframe and took the first result. There is massive room for improvement in this technique, but from a preliminary skimming it looks like it worked well enough

This works fairly decently, but is very slow. 
It should be looked at again when the ingredient model is cleaned up. 

Some notes:
    - It handles plural very poorly. ie carrot vs carrots give very  different results
    - The code is a bit of a mess, and needs to cleaned up given time
    - it would be beneficial to link the weights to the products as well. 
    - this new table should be split into 2. There is a lot of redundant info that can be stored better

In [1]:
import pandas as pd
import numpy as np

In [2]:
food_desc = pd.read_csv('FOOD_DESC.csv')
food_group = pd.read_csv('FD_GROUP.csv')
langdesc = pd.read_csv('LANGDESC.csv')
langual = pd.read_csv('LANGUAL.csv')
weight = pd.read_csv('WEIGHT.csv')

In [3]:
master = pd.merge(food_desc, food_group, on='FdGrp_Cd', how='left')

In [6]:
langs = pd.merge(langual, langdesc, on='Factor', how='left')

In [8]:
langs = langs[langs.Factor.str.startswith('B')]

In [9]:
mtr_lng = pd.merge(langs, master[['NDB_No', 'Long_Desc']], on='NDB_No', how='left')

In [10]:
master = pd.merge(master, mtr_lng[['NDB_No','Description']], on='NDB_No', how='outer')

In [11]:
master.shape

(7793, 16)

In [12]:
master.loc[:, 'FdGrp_desc'].value_counts()


Beef Products                          954
Vegetables and Vegetable Products      814
Baked Products                         517
Lamb, Veal, and Game Products          464
Poultry Products                       383
Beverages                              366
Sweets                                 358
Fruits and Fruit Juices                355
Baby Foods                             345
Pork Products                          336
Fast Foods                             312
Dairy and Egg Products                 291
Legumes and Legume Products            290
Finfish and Shellfish Products         264
Soups, Sauces, and Gravies             254
Fats and Oils                          216
Breakfast Cereals                      195
Cereal Grains and Pasta                181
Snacks                                 176
Sausages and Luncheon Meats            167
American Indian/Alaska Native Foods    165
Nut and Seed Products                  137
Restaurant Foods                       109
Meals, Entr

In [14]:
merged_weight = pd.merge(weight, master[['NDB_No', 'Long_Desc']], on='NDB_No')

In [15]:
master.head()

,NDB_No,FdGrp_Cd,Long_Desc,Shrt_Desc,Com_Name,ManufacName,Survey,Ref_Desc,Refuse,Sci_Name,N_FActor,Pro_Factor_,Fat_Factor_,CHO_Factor,FdGrp_desc,Description
0,1001,100,"Butter, salted","BUTTER,WITH SALT",NaN,NaN,Y,NaN,0.0,NaN,6.38,4.27,8.79,3.87,Dairy and Egg Products,NaN
1,1002,100,"Butter, whipped, with salt","BUTTER,WHIPPED,W/ SALT",NaN,NaN,Y,NaN,0.0,NaN,6.38,NaN,NaN,NaN,Dairy and Egg Products,NaN
2,1003,100,"Butter oil, anhydrous","BUTTER OIL,ANHYDROUS",NaN,NaN,Y,NaN,0.0,NaN,6.38,4.27,8.79,3.87,Dairy and Egg Products,NaN
3,1004,100,"Cheese, blue","CHEESE,BLUE",NaN,NaN,Y,NaN,0.0,NaN,6.38,4.27,8.79,3.87,Dairy and Egg Products,NaN
4,1005,100,"Cheese, brick","CHEESE,BRICK",NaN,NaN,Y,NaN,0.0,NaN,6.38,4.27,8.79,3.87,Dairy and Egg Products,NaN


In [16]:
from fuzzywuzzy import process, fuzz

In [17]:
choices = master.Description.values
choices2 = master.Shrt_Desc.values

In [18]:
ing = "button mushrooms"
process.extract(ing, choices, scorer=fuzz.token_set_ratio )

[('CULTIVATED MUSHROOM', 69),
 ('CULTIVATED MUSHROOM', 69),
 ('CULTIVATED MUSHROOM', 69),
 ('CULTIVATED MUSHROOM', 69),
 ('CULTIVATED MUSHROOM', 69)]

In [19]:
process.extract(ing, choices2, scorer=fuzz.token_set_ratio, limit=140)

[('MUSHROOMS,SHIITAKE,RAW', 72),
 ('MUSHROOMS,CHANTERELLE,RAW', 72),
 ('MUSHROOMS,MOREL,RAW', 72),
 ('MUSHROOMS,PORTABELLA,GRILLED', 72),
 ('MUSHROOMS,WHITE,RAW', 72),
 ('MUSHROOMS,WHITE,CKD,BLD,DRND,WO/ SALT', 72),
 ('MUSHROOMS,WHITE,STIR-FRIED', 72),
 ('MUSHROOMS,CND,DRND SOL', 72),
 ('MUSHROOMS,PORTABELLA,RAW', 72),
 ('MUSHROOMS,BROWN,ITALIAN,OR CRIMINI,RAW', 72),
 ('MUSHROOMS,SHIITAKE,STIR-FRIED', 72),
 ('MUSHROOMS,SHIITAKE,DRIED', 72),
 ('MUSHROOMS,SHIITAKE,CKD,WO/SALT', 72),
 ('TOMATO PRODUCTS,CND,SAU,W/MUSHROOMS', 72),
 ('MUSHROOMS,WHITE,CKD,BLD,DRND,W/ SALT', 72),
 ('MUSHROOMS,SHIITAKE,CKD,W/SALT', 72),
 ('MUSHROOMS,BROWN,ITALIAN,OR CRIMINI,EXPOSED TO UV LT,RAW', 72),
 ('MUSHROOMS,PORTABELLA,EXPOSED TO UV LT,GRILLED', 72),
 ('MUSHROOMS,ENOKI,RAW', 72),
 ('MUSHROOMS,OYSTER,RAW', 72),
 ('MUSHROOMS,STRAW,CND,DRND SOL', 72),
 ('MUSHROOMS,WHITE,MICROWAVED', 72),
 ('MUSHROOMS,MAITAKE,RAW', 72),
 ('MUSHROOMS,PORTABELLA,EXPOSED TO ULTRAVIOLET LT,RAW', 72),
 ('SOUP,BF MUSHROOM,CND,COND'

In [20]:
master[master.Description == 'MUSHROOM']

,NDB_No,FdGrp_Cd,Long_Desc,Shrt_Desc,Com_Name,ManufacName,Survey,Ref_Desc,Refuse,Sci_Name,N_FActor,Pro_Factor_,Fat_Factor_,CHO_Factor,FdGrp_desc,Description
2625,11261,1100,"Mushrooms, white, cooked, boiled, drained, wit...","MUSHROOMS,WHITE,CKD,BLD,DRND,WO/ SALT",NaN,NaN,Y,NaN,0.0,Agaricus bisporus,6.25,2.62,8.37,3.48,Vegetables and Vegetable Products,MUSHROOM
2627,11264,1100,"Mushrooms, canned, drained solids","MUSHROOMS,CND,DRND SOL",NaN,NaN,Y,NaN,0.0,NaN,6.25,2.62,8.37,3.48,Vegetables and Vegetable Products,MUSHROOM
3024,11797,1100,"Mushrooms, white, cooked, boiled, drained, wit...","MUSHROOMS,WHITE,CKD,BLD,DRND,W/ SALT",NaN,NaN,NaN,NaN,0.0,Agaricus bisporus,6.25,2.62,8.37,3.48,Vegetables and Vegetable Products,MUSHROOM


In [21]:
master.shape

(7793, 16)

So my general strategy for linking this list with my database will be too:
    - check scores from pattern matching using fuzzy from B column of linual and
      the Long_Desc from FOOD_DESC
    - of those two scores, take which ever is the highest 
    - if several are highest take an average of the values I want or majority take all
    - look up item on the table and use the values I want (food group, Description, NDB_No for 
      adding measurement values later)

In [22]:
import asyncio
import asyncpg

In [23]:
async def run(table):
    conn = await asyncpg.connect(user='postgres', password='postgres',
                                 port='5435',
                                 database='recipes_test', host='localhost')
    values = await conn.fetch(f'''SELECT * FROM {table}''')
    await conn.close()
    return values

In [24]:
loop = asyncio.get_event_loop()

In [25]:
z = loop.create_task(run('ingredient_list'))

In [26]:
z.result()

[<Record ingredient='tomatoes' id=1>,
 <Record ingredient='kosher salt' id=2>,
 <Record ingredient='red onion' id=3>,
 <Record ingredient='green bell pepper' id=4>,
 <Record ingredient='red bell pepper' id=5>,
 <Record ingredient='yellow bell pepper' id=6>,
 <Record ingredient='cucumber' id=7>,
 <Record ingredient='watermelon gelatin' id=8>,
 <Record ingredient='extra - virgin olive oil' id=9>,
 <Record ingredient='water' id=10>,
 <Record ingredient='basil' id=11>,
 <Record ingredient='cool whip' id=12>,
 <Record ingredient='seedless watermelon' id=13>,
 <Record ingredient='elbow macaroni' id=14>,
 <Record ingredient='graham cracker crust' id=15>,
 <Record ingredient='penne' id=16>,
 <Record ingredient='american cheese' id=17>,
 <Record ingredient='flagship cheese sauce' id=18>,
 <Record ingredient='celery' id=19>,
 <Record ingredient='cheddar' id=20>,
 <Record ingredient='green pepper' id=21>,
 <Record ingredient='gruyere cheese' id=22>,
 <Record ingredient='pimento' id=23>,
 <Record 

In [27]:
res = z.result()

In [28]:
res[0]['ingredient']

'tomatoes'

In [29]:
ing_list = [i['ingredient'] for i in res]

In [30]:
f = loop.create_task(run('ingredients'))

In [31]:
res = f.result()

InvalidStateError: Result is not set.

In [32]:
orig_ing = [i['original_text'] for i in res]

KeyError: 'original_text'

In [33]:
orig_ing

NameError: name 'orig_ing' is not defined

In [34]:
process.extract('Lentils, mature seeds, cooked, boiled, with salt',
               orig_ing, scorer=fuzz.token_set_ratio)

NameError: name 'orig_ing' is not defined

In [35]:
def filter_ingredients(ingredient):
    """
        this should just be a dictionary
    """
    
    if ingredient in ['sugar','sugars', 'coarse sugar']:
        return 'sugars powdered'
    elif ingredient == 'milk':
        return 'MILK,WHL,3.25% MILKFAT'
    elif ingredient == 'extra - virgin olive oil':
        return 'olive oil'
    elif ingredient == 'water':
        return 'WATER,BTLD,GENERIC'
    elif ingredient in ['penne', 'elbow macaroni']:
        return 'PASTA,DRY,ENR'
    elif ingredient == 'eggs':
        return 'EGG,WHL,RAW,FRSH'
    elif ingredient == 'vanilla':
        return 'VANILLA EXTRACT'
    elif ingredient == 'flour':
        return 'WHEAT,DURUM'
    elif ingredient == 'brown sugar':
        return 'SUGARS,BROWN'
    elif ingredient == 'salt' or ingredient == 'kosher salt':
        return 'SALT,TABLE'
    elif ingredient == 'pepper' or ingredient == 'salt and pepper':
        return 'PEPPER,BLACK'
    elif ingredient == 'lemon juice':
        return 'LEMON JUICE, RAW'
    elif ingredient in ['potato', 'potatoes']:
        return 'POTATOES,FLESH & SKN,RAW'
    elif ingredient == 'beef':
        return 'BEEF,GRASS-FED,GROUND,RAW'
    elif ingredient == 'salmon':
        return 'SALMON,ATLANTIC,WILD,RAW'
    elif ingredient in ['tomatoes', 'tomato']:
        return 'TOMATOES,RED,RIPE,RAW,YEAR RND AVERAGE'
    elif ingredient == 'carrots':
        return 'carrot'
    elif ingredient == 'bananas':
        return 'banana'
    elif ingredient == 'peaches':
        return 'peach'
    elif ingredient == 'button mushrooms':
        return 'button mushroom'
    elif ingredient == 'peas':
        return 'pea'
    else:
        return ingredient

In [37]:
def compare_string(st, choice_a, choice_b):
    st = filter_ingredients(st)
    a = process.extract(st, choice_a, scorer=fuzz.token_set_ratio)
    b = process.extract(st, choice_b, scorer=fuzz.token_set_ratio)
    
    if a:
        top_score_a = a[0][1]
        print(f'top a: {a[0][0]} with a score of {a[0][1]}')
    else:
        top_score_a = 0
    
    if b:
        top_score_b = b[0][1]
        print(f'top b: {b[0][0]} with a score of {b[0][1]}')
    else:
        top_score_b = 0
        
    
    if top_score_a >= top_score_b:
        if top_score_a > 75:
            selection = a[0][0]
            return ('a', selection, top_score_a)
        else:
            return None
    else:
        if top_score_b > 75:
            selection = b[0][0]
            return ('b', selection, top_score_b)
        else:
            return None
    
    

In [38]:
for i in ing_list[150:300]:
    print(i)
    compare_string(i, choices, choices2)
    print('\n')

whipped cream
top a: ROSEAPPLE with a score of 55
top b: CREAM,WHIPPED,CRM TOPPING,PRESSURIZED with a score of 100


rice wrappers
top a: RICE with a score of 100
top b: RICE CRACKERS with a score of 77


halibut
top a: SHALLOT with a score of 57
top b: HALIBUT,ATLANTIC&PACIFIC,RAW with a score of 100


foie gras mousse
top a: IRISH MOSS with a score of 62
top b: PATE DE FOIE GRAS,CND (GOOSE LIVER PATE),SMOKED with a score of 72


chive batons
top a: CHIVE with a score of 100
top b: ACORNS,DRIED with a score of 58


chives
top a: CHIVE with a score of 91
top b: CHIVES,RAW with a score of 100


truffle oil
top a: PILI TREE with a score of 60
top b: PATE  TRUFFLE FLAVOR with a score of 78


foie gras
top a: FIELD PEA with a score of 56
top b: PATE DE FOIE GRAS,CND (GOOSE LIVER PATE),SMOKED with a score of 100


heavy cream
top a: CARAWAY with a score of 56
top b: CHEESE,CREAM with a score of 70


white truffle oil
top a: FIELD CORN, WHITE with a score of 61
top b: TUNA,WHITE,CND IN OIL,D

top a: ACORN SQUASH with a score of 100
top b: SQUASH,WINTER,ACORN,RAW with a score of 100


black beans
top a: BLACK GRAM BEAN with a score of 77
top b: BEANS,BLACK,MATURE SEEDS,RAW with a score of 100


sea salt and pepper
top a: HOT PEPPER with a score of 75
top b: PEPPER,BLACK with a score of 67


avocado
top a: AVOCADO with a score of 100
top b: OIL,AVOCADO with a score of 100


fryer
top a: BROILER OR FRYER CHICKEN with a score of 100
top b: CHICKEN,BROILER OR FRYER,MEAT&SKN&GIBLETS&NECK,FRIED,BATTER with a score of 100


wasabi paste
top a: WASABI with a score of 100
top b: WASABI with a score of 100


tuna
top a: TURNIP with a score of 60
top b: TUNA,FRESH,BLUEFIN,RAW with a score of 100


bamboo skewers
top a: BAMBOO with a score of 100
top b: BABYFOOD,COOKIES with a score of 60


lemon pepper seasoning
top a: LEMON with a score of 100
top b: TURKEY,BREAST,SMOKED,LEMON PEPPER FLAVOR,97% FAT-FREE with a score of 71


swiss cheese
top a: SWISS CHARD with a score of 70
top b: CHE

In [39]:
def parseIng(ingredient, df, desc, short_desc):
    """
        this is brutally slow
    """
    rows_index = compare_string(ingredient, desc, short_desc)
    if rows_index:
        if rows_index[0] == 'a':
            rows = df[df.Description == rows_index[1]]
        elif rows_index[0] == 'b':
            rows = df[df.Shrt_Desc == rows_index[1]]
    else:
        return None
    item = rows.iloc[0]
    return item[['FdGrp_Cd', 'NDB_No', 'FdGrp_desc', 'Shrt_Desc']]

In [43]:
async def select_all(table):
    conn = await asyncpg.connect(user='postgres', password='postgres',
                                 port='5435',
                                 database='recipes_test', host='localhost')
    values = await conn.fetch(f'''SELECT * FROM {table}''')
    await conn.close()
    return values

In [44]:
async def create_table():
    conn = await asyncpg.connect(user='postgres', password='postgres',
                                 port='5435',database='recipes_test', host='localhost')

    await conn.execute("""
        CREATE TABLE ingredient_information (
            id INTEGER NOT NULL,
            NBD_No INTEGER,
            FdGroup_num INTEGER,
            FdGroup_name TEXT,
            Shrt_Desc TEXT,
            PRIMARY KEY (id),
            FOREIGN KEY (id) REFERENCES ingredient_list (id)
        )
    """)
    await conn.close()
    

In [45]:
loop.create_task(create_table())

<Task pending coro=<create_table() running at <ipython-input-44-5f59d6840d3a>:1>>

In [46]:
async def stock_ingredient_info(df, desc, short_desc):
    conn = await asyncpg.connect(user='postgres', password='postgres',
                                 port='5435',database='recipes_test', host='localhost')
    
    print('getting ingredient list')
    ing_list = await select_all('ingredient_list')
    print('have ingredient list')
    for ing in ing_list:
        
        ingId = ing['id']
        ingName = ing['ingredient']
        print(f'checking ingredient {ingName} {ingId}')

        
        info = parseIng(ingName, df, desc, short_desc)
        if info is not None:
            print(f'putting {ingName} into database')
            # print(f'id: {str(ingId)}, info[nbd]: {str(info["NBD_No"])}')
            NBD = info['NDB_No']
            fd_grp_num = info['FdGrp_Cd']
            fd_grp_desc = info['FdGrp_desc']
            shrt_Desc = info['Shrt_Desc']
            print(f'{NBD}, {fd_grp_num}, {fd_grp_desc}, {shrt_Desc}')
            await conn.execute("""
                INSERT INTO ingredient_information(
                    id,
                    NBD_No,
                    FdGroup_num,
                    FdGroup_name,
                    Shrt_Desc
                ) VALUES ( $1, $2, $3, $4, $5);
            """, ingId, NBD, fd_grp_num, fd_grp_desc, shrt_Desc)
            print('item in database')
    await conn.close()
    print('conn closed')

In [47]:
z = loop.create_task(stock_ingredient_info(master, choices, choices2))

In [48]:
z.result()

InvalidStateError: Result is not set.

getting ingredient list
have ingredient list
checking ingredient tomatoes 1
top a: RED CURRANT with a score of 43
top b: TOMATOES,RED,RIPE,RAW,YEAR RND AVERAGE with a score of 100
putting tomatoes into database
11529, 1100, Vegetables and Vegetable Products, TOMATOES,RED,RIPE,RAW,YEAR RND AVERAGE
item in database
checking ingredient kosher salt 2
top a: CATTLE with a score of 62
top b: SALT,TABLE with a score of 100
putting kosher salt into database
2047, 200, Spices and Herbs, SALT,TABLE
item in database
checking ingredient red onion 3
top a: ONION with a score of 100
top b: ONIONS,RAW with a score of 74
putting red onion into database
2026, 200, Spices and Herbs, ONION POWDER
item in database
checking ingredient green bell pepper 4
top a: GREEN BELL PEPPER with a score of 100
top b: PEPPER,BLACK with a score of 67
putting green bell pepper into database
11333, 1100, Vegetables and Vegetable Products, PEPPERS,SWT,GRN,RAW
item in database
checking ingredient red bell pepper 5
top a: RE

top a: GARLIC with a score of 100
top b: GARLIC POWDER with a score of 100
putting garlic powder into database
2020, 200, Spices and Herbs, GARLIC POWDER
item in database
checking ingredient  35
top a: nan with a score of 0
top b: BUTTER,WITH SALT with a score of 0
checking ingredient butter 36
top a: BUTTER BUR with a score of 100
top b: BUTTER,WITH SALT with a score of 100
putting butter into database
11106, 1100, Vegetables and Vegetable Products, BUTTERBUR,(FUKI),RAW
item in database
checking ingredient potatoes 37
top a: SWEET POTATO with a score of 47
top b: POTATOES,FLESH & SKN,RAW with a score of 100
putting potatoes into database
11352, 1100, Vegetables and Vegetable Products, POTATOES,FLESH & SKN,RAW
item in database
checking ingredient kernel corn 38
top a: CORN with a score of 100
top b: CORN,SWT,YEL,CND,WHL KERNEL,DRND SOL with a score of 100
putting kernel corn into database
20030, 2000, Cereal Grains and Pasta, HOMINY,CANNED,WHITE
item in database
checking ingredient che

top a: CORIANDER with a score of 50
top b: PORK  FRSH  LOIN  TENDERLOIN   LN ONLY  RAW with a score of 100
putting pork tenderloin into database
10060, 1000, Pork Products, PORK  FRSH  LOIN  TENDERLOIN   LN ONLY  RAW
item in database
checking ingredient salmon 74
top a: WILD RICE with a score of 62
top b: SALMON,ATLANTIC,WILD,RAW with a score of 100
putting salmon into database
15076, 1500, Finfish and Shellfish Products, SALMON,ATLANTIC,WILD,RAW
item in database
checking ingredient fennel bulbs 75
top a: FENNEL with a score of 100
top b: FENNEL,BULB,RAW with a score of 81
putting fennel bulbs into database
2018, 200, Spices and Herbs, FENNEL SEED
item in database
checking ingredient extra virgin olive oil 76
top a: BLACK OLIVE with a score of 62
top b: OIL,OAT with a score of 60
checking ingredient olive oil 77
top a: BLACK OLIVE with a score of 71
top b: OIL,OLIVE,SALAD OR COOKING with a score of 100
putting olive oil into database
4053, 400, Fats and Oils, OIL,OLIVE,SALAD OR COOKING

top a: TOMATO with a score of 100
top b: SAUCE,TOMATO CHILI SAU,BTLD,W/SALT with a score of 100
putting tomato sauce into database
11527, 1100, Vegetables and Vegetable Products, TOMATOES,GREEN,RAW
item in database
checking ingredient chili powder 114
top a: CAULIFLOWER with a score of 61
top b: CHILI POWDER with a score of 100
putting chili powder into database
2009, 200, Spices and Herbs, CHILI POWDER
item in database
checking ingredient corn 115
top a: VEGETABLE CORN with a score of 100
top b: BABYFOOD,VEG,BUTTERNUT SQUASH&CORN with a score of 100
putting corn into database
11167, 1100, Vegetables and Vegetable Products, CORN,SWT,YEL,RAW
item in database
checking ingredient white onion 116
top a: ONION with a score of 100
top b: CORN GRAIN,WHITE with a score of 74
putting white onion into database
2026, 200, Spices and Herbs, ONION POWDER
item in database
checking ingredient mexican - style corn 117
top a: CORN with a score of 100
top b: CORN,SWT,YEL,CND,CRM STYLE,REG PK with a scor

...

top a: GRAPE, CORINTH with a score of 45
top b: SOUP,BOUILLON CUBES&GRANULES,LO NA,DRY with a score of 58
checking ingredient awase miso 12295
top a: MAITAKE MUSHROOM with a score of 54
top b: MISO with a score of 100
putting awase miso into database
16112, 1600, Legumes and Legume Products, MISO
item in database
checking ingredient cheesecake timbales 12301
top a: CHINESE KALE with a score of 58
top b: CHEESE,TILSIT with a score of 69
checking ingredient unsalted macadamia nuts 12302
top a: MACADAMIA with a score of 100
top b: MACADAMIA NUTS,RAW with a score of 88
putting unsalted macadamia nuts into database
12131, 1200, Nut and Seed Products, MACADAMIA NUTS,RAW
item in database
checking ingredient g pasta shells 12303
top a: CASABA MELON with a score of 46
top b: TOSTADA SHELLS,CORN with a score of 67
checking ingredient caraway 12337
top a: CARAWAY with a score of 100
top b: CHEESE,CARAWAY with a score of 100
putting caraway into database
2005, 200, Spices and Herbs, CARAWAY SEED
i

top a: SOUR CHERRY with a score of 52
top b: Beef, shoulder steak, boneless, lean only, 0", choice, grill with a score of 80
putting ham shoulder into database
13943, 1300, Beef Products, Beef, shoulder steak, boneless, lean only, 0", choice, grill
item in database
checking ingredient breadcrumbs bread 12528
top a: BROAD BEAN with a score of 59
top b: BREAD,EGG with a score of 71
checking ingredient mozzarella cheese sticks 12543
top a: CHINESE PRESERVING MELON with a score of 50
top b: CHEESE SUB,MOZZARELLA with a score of 89
putting mozzarella cheese sticks into database
1161, 100, Dairy and Egg Products, CHEESE SUB,MOZZARELLA
item in database
checking ingredient canschopped clams 12545
top a: ANCHO PEPPER with a score of 55
top b: HAM,CHOPPED,CANNED with a score of 74
checking ingredient wheat bran flake cereal 12549
top a: WHEAT with a score of 100
top b: WHEAT BRAN,CRUDE with a score of 77
putting wheat bran flake cereal into database
20012, 2000, Cereal Grains and Pasta, BULGUR,D

top a: CABBAGE with a score of 67
top b: CRABAPPLES,RAW with a score of 64
checking ingredient fennel bulb anise 12556
top a: ANISE with a score of 100
top b: FENNEL,BULB,RAW with a score of 85
putting fennel bulb anise into database
2002, 200, Spices and Herbs, ANISE SEED
item in database
checking ingredient beef back ribs 12564
top a: PEPPER, BLACK with a score of 62
top b: BEEF,RIB,BACK RIBS,BONE-IN,LN,0" FAT,ALL GRDS,CKD,BRSD with a score of 100
putting beef back ribs into database
23180, 1300, Beef Products, BEEF,RIB,BACK RIBS,BONE-IN,LN,0" FAT,ALL GRDS,CKD,BRSD
item in database
checking ingredient florida man's bar b que sauce 12566
top a: CALIFORNIA VALENCIA ORANGE with a score of 47
top b: SAUCE,BARBECUE,SWT BABY RAY'S,ORIGINAL with a score of 63
checking ingredient veal chops 12571
top a: COWPEA with a score of 62
top b: PORK,FRSH,LOIN,BLADE (CHOPS OR ROASTS),BONE-IN,LN&FAT,RAW with a score of 67
checking ingredient summer squash gratin 12573
top a: SUMMER SQUASH with a score 

top a: ROSELLE with a score of 62
top b: EGG,WHOLE,COOKED,OMELET with a score of 100
putting omelet into database
1130, 100, Dairy and Egg Products, EGG,WHOLE,COOKED,OMELET
item in database
checking ingredient denbu 12582
top a: PEANUT with a score of 55
top b: NUTS,PINE NUTS,DRIED with a score of 40
checking ingredient rollo candies 12585
top a: CHANTERELLE with a score of 58
top b: CANDIES,HARD with a score of 74
checking ingredient unsweetened grape juice 12586
top a: GRAPE with a score of 100
top b: CRANBERRY JUC,UNSWTND with a score of 68
putting unsweetened grape juice into database
2068, 200, Spices and Herbs, VINEGAR,RED WINE
item in database
checking ingredient demi - glace 12589
top a: MACE with a score of 57
top b: DULCE DE LECHE with a score of 50
checking ingredient & garlic cheese 12591
top a: GARLIC with a score of 100
top b: CHEESE,BRICK with a score of 80
putting & garlic cheese into database
2020, 200, Spices and Herbs, GARLIC POWDER
item in database
checking ingredie

top a: TURNIP with a score of 44
top b: Catsup with a score of 44
checking ingredient asian sweet potato 12641
top a: POTATO with a score of 100
top b: SWEET POTATO LEAVES,RAW with a score of 80
putting asian sweet potato into database
11352, 1100, Vegetables and Vegetable Products, POTATOES,FLESH & SKN,RAW
item in database
checking ingredient processed cheesed 12642
top a: CHINESE CHESTNUT with a score of 55
top b: CHEESE,CHESHIRE with a score of 69
checking ingredient tapenade butter 12646
top a: BUTTER BUR with a score of 75
top b: PEANUT BUTTER,RED NA with a score of 69
checking ingredient dry coconut powder 12567
top a: COCONUT with a score of 100
top b: CURRY POWDER with a score of 73
putting dry coconut powder into database
12104, 1200, Nut and Seed Products, COCONUT MEAT,RAW
item in database
checking ingredient sweet milk 12568
top a: BASIL, SWEET with a score of 76
top b: WHEY,SWEET,FLUID with a score of 67
putting sweet milk into database
2003, 200, Spices and Herbs, SPICES,B

In [251]:
f = loop.create_task(select_all('ingredient_list'))

In [254]:
z = f.result()

In [256]:
z[0]['id']

1

In [257]:
z[0]['ingredient']

'tomatoes'